# memstack-agent-ui WebSocket 客户端

本 notebook 演示如何使用 @memstack-agent-ui/sdk 的 WebSocket 客户端。

## 概述

WebSocketClient 提供：
- 自动重连（指数退避）
- 心跳保持连接
- 事件路由到处理器
- 多会话支持（多标签页）

## 1. 创建 WebSocket 客户端

In [ ]:
# 基本用法

basic_usage = '''
import { WebSocketClient } from '@memstack-agent-ui/sdk';

// 创建客户端
const client = new WebSocketClient({
  url: 'ws://localhost:8000/agent/ws',
  token: 'your-auth-token', // 可选
  conversationId: 'conv-123', // 可选
  
  // 连接配置
  heartbeatInterval: 30000, // 30秒心跳
  maxReconnectAttempts: 5, // 最大重连次数
  reconnectDelay: 1000, // 初始重连延迟
  
  // 回调
  onStatusChange: (status) => {
    console.log('Connection status:', status);
  },
  onError: (error) => {
    console.error('Connection error:', error);
  },
});

// 连接
await client.connect();
console.log('Connected!');
// 会话 ID 用于多标签页支持
console.log('Session ID:', client.sessionId);
'''

print(basic_usage)

## 2. 订阅事件

In [ ]:
# 订阅特定事件类型

subscribe_events = '''
// 订阅特定事件类型
const unsubscribeThought = client.on('thought', (event) => {
  console.log('Agent thought:', event.data.content);
});

const unsubscribeComplete = client.on('complete', (event) => {
  console.log('Agent completed:', event.data);
});

// 取消订阅
unsubscribeThought();
unsubscribeComplete();
'''

print(subscribe_events)

## 3. 订阅所有事件

In [ ]:
# 订阅所有事件（用于日志/调试）

subscribe_all = '''
// 订阅所有事件
const unsubscribeAll = client.onAny((event) => {
  console.log(`[${event.type}]`, event.data);
});

// 取消订阅
unsubscribeAll();
'''

print(subscribe_all)

## 4. 发送消息

In [ ]:
# 发送消息到 Agent

send_message = '''
// 发送聊天消息
const sent = client.send({
  type: 'chat',
  conversation_id: 'conv-123',
  message: 'Hello, agent!',
  file_ids: ['file-001'], // 可选的文件附件
});

if (sent) {
  console.log('Message sent successfully');
} else {
  console.log('Failed to send - not connected');
}
'''

print(send_message)

## 5. 监控连接状态

In [ ]:
# 监控连接状态变化

monitor_status = '''
import type { WebSocketStatus } from '@memstack-agent-ui/sdk';

// 订阅状态变化
const unsubscribeStatus = client.onStatusChange((status: WebSocketStatus) => {
  switch (status) {
    case 'connecting':
      console.log('Connecting to server...');
      break;
    case 'connected':
      console.log('Connected!');
      break;
    case 'disconnected':
      console.log('Disconnected');
      break;
    case 'error':
      console.log('Connection error');
      break;
  }
});

// 获取当前状态
const currentStatus = client.getStatus();
console.log('Current status:', currentStatus);

// 检查是否已连接
if (client.isConnected()) {
  console.log('Client is connected');
}
'''

print(monitor_status)

## 6. 断开连接

In [ ]:
# 正确断开连接

disconnect = '''
// 手动断开（停止自动重连）
client.disconnect();
console.log('Disconnected');

// 注意：disconnect() 会：
// 1. 停止心跳
// 2. 清除重连定时器
// 3. 关闭 WebSocket
// 4. 设置状态为 'disconnected'
'''

print(disconnect)

## 7. 自动重连机制

In [ ]:
# 自动重连使用指数退避

reconnect_mechanism = '''
// 重连行为：
// - 连接断开时自动尝试重连
// - 使用指数退避：1s -> 2s -> 4s -> 8s -> 16s
// - 达到最大重连次数后停止

const client = new WebSocketClient({
  url: 'ws://localhost:8000/agent/ws',
  maxReconnectAttempts: 5,
  reconnectDelay: 1000, // 初始延迟 1s
});

// 重连时间线（假设持续失败）:
// 尝试 1: 等待 1s 后重连
// 尝试 2: 等待 2s 后重连
// 尝试 3: 等待 4s 后重连
// 尝试 4: 等待 8s 后重连
// 尝试 5: 等待 16s 后重连
// 放弃重连
'''

print(reconnect_mechanism)

## 8. 心跳机制

In [ ]:
# 心跳保持连接活跃

heartbeat = '''
// 默认每 30 秒发送一次 ping
// 如果连接断开，心跳会自动停止
// 重连成功后，心跳会自动恢复

const client = new WebSocketClient({
  url: 'ws://localhost:8000/agent/ws',
  heartbeatInterval: 30000, // 30 秒
});

// 心跳消息格式:
// { "type": "ping" }

// 服务器应响应 pong
// 如果超时无响应，可能触发重连
'''

print(heartbeat)

## 9. 多标签页支持

In [ ]:
# 每个客户端有唯一的 session ID

multi_tab = '''
const client1 = new WebSocketClient({ url: 'ws://localhost:8000/agent/ws' });
const client2 = new WebSocketClient({ url: 'ws://localhost:8000/agent/ws' });

console.log('Client 1 session:', client1.sessionId);
console.log('Client 2 session:', client2.sessionId);
// 两个不同的 UUID

// session_id 会作为查询参数发送:
// ws://localhost:8000/agent/ws?session_id=xxx-xxx-xxx

// 服务器可以使用 session_id 区分不同标签页的连接
'''

print(multi_tab)

## 10. 完整示例

In [ ]:
# 完整的聊天客户端示例

complete_example = '''
import { WebSocketClient } from '@memstack-agent-ui/sdk';
import type { WebSocketStatus, AgentEvent } from '@memstack-agent-ui/sdk';

class AgentChatClient {
  private client: WebSocketClient;
  private conversationId: string;

  constructor(wsUrl: string, conversationId: string, token?: string) {
    this.conversationId = conversationId;
    
    this.client = new WebSocketClient({
      url: wsUrl,
      token,
      conversationId,
      onStatusChange: this.handleStatusChange.bind(this),
      onError: this.handleError.bind(this),
    });

    // 订阅事件
    this.setupEventHandlers();
  }

  private setupEventHandlers(): void {
    // 思考事件
    this.client.on('thought', (event) => {
      console.log('🤔 Thinking:', event.data);
    });

    // 工具调用事件
    this.client.on('act', (event) => {
      console.log('🔧 Tool call:', event.data.tool_name);
    });

    this.client.on('observe', (event) => {
      console.log('👀 Tool result:', event.data.result);
    });

    // 文本流事件
    this.client.on('text_delta', (event) => {
      process.stdout.write(event.data.delta);
    });

    // 完成事件
    this.client.on('complete', (event) => {
      console.log('\n✅ Complete!');
    });

    // 错误事件
    this.client.on('error', (event) => {
      console.error('❌ Error:', event.data.message);
    });
  }

  private handleStatusChange(status: WebSocketStatus): void {
    console.log('📡 Status:', status);
  }

  private handleError(error: Error): void {
    console.error('🚨 Connection error:', error);
  }

  async connect(): Promise<void> {
    await this.client.connect();
  }

  send(message: string): boolean {
    return this.client.send({
      type: 'chat',
      conversation_id: this.conversationId,
      message,
    });
  }

  disconnect(): void {
    this.client.disconnect();
  }
}

// 使用
const chat = new AgentChatClient(
  'ws://localhost:8000/agent/ws',
  'conv-123',
  'your-token'
);

await chat.connect();
chat.send('Hello, agent!');
// ... 等待响应
chat.disconnect();
'''

print(complete_example)

## 总结

本 notebook 演示了 WebSocketClient 的核心功能：

1. **连接管理** - connect(), disconnect(), isConnected()
2. **事件订阅** - on(), onAny(), 取消订阅
3. **消息发送** - send() 方法
4. **状态监控** - getStatus(), onStatusChange()
5. **自动重连** - 指数退避机制
6. **心跳保持** - 30 秒间隔
7. **多标签页** - 唯一 session ID

下一步：查看 `fe-03-react-hooks.ipynb` 学习 React Hooks 使用。